In [2]:
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

In [3]:
device = torch.device("cuda:0")

### Extract, Transform, Load

In [4]:
# Extract and Transform
train_set = torchvision.datasets.MNIST(
    root=r'./Dataset',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

test_set = torchvision.datasets.MNIST(
    root=r'./Dataset',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

# Load
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size = 100,
    shuffle = True  # to have the data reshuffled at every epoch
)

test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size = 100,
    shuffle = False
)

In [5]:
batch = next(iter(train_loader))
images, labels = batch
print(images.shape,'\t',labels.shape)

torch.Size([100, 1, 28, 28]) 	 torch.Size([100])


### Create Network

In [6]:
class ConvNetwork(nn.Module):
    def __init__(self):
        super(ConvNetwork, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3, stride=1, padding=1)

        self.fc1 = nn.Linear(in_features=12*6*6, out_features=120)
        # self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=120, out_features=10)
        
    def forward(self, d):
        # conv1 layer
        d = self.conv1(d)
        d = nn.functional.relu(d)
        d = nn.functional.max_pool2d(d, kernel_size=2, stride=2)
    
        # conv2 layer
        d = self.conv2(d)
        d = nn.functional.relu(d)
        d = nn.functional.max_pool2d(d, kernel_size=2, stride=2)

        # fully connected Layer1
        # d = d.reshape(-1, 12*4*4)
        d = d.flatten(start_dim=1)
        d = self.fc1(d)
        d = nn.functional.relu(d)

        # fully connected Layer2
        # d = self.fc2(d)
        # d = nn.functional.relu(d)

        # output layer
        d = self.out(d)
        # d = nn.functional.softmax(d)

        return d

In [7]:
network = ConvNetwork()
network.to(device)

ConvNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=432, out_features=120, bias=True)
  (out): Linear(in_features=120, out_features=10, bias=True)
)

### Activate TensorBoard

In [9]:
%load_ext tensorboard
%tensorboard --logdir ./runs/ --port=6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 30348), started 0:01:24 ago. (Use '!kill 30348' to kill it.)

### Train

In [10]:
import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter

In [11]:
def test_model(model, loss_fn, test_loader, device=torch.device("cpu")):
    model.eval()

    loss = 0.
    num_correct = 0.
    
    for batch_idx, (x, y) in enumerate(test_loader):
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss += loss_fn(y_pred, y).item()
        _, predicted = torch.max(y_pred.data, 1)
        num_correct += (predicted == y).sum().item()

    model.train()
    
    loss /= len(test_loader.dataset)
    num_correct /= len(test_loader.dataset)
    
    return loss, num_correct

In [12]:
def get_num_correct(preds, labels):
    '''
    Calculate the number of correct predictions
    '''
    return preds.argmax(dim=1).eq(labels).sum().item()

In [14]:
# Learning Rate
lr = 0.008

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(network.parameters(), lr=lr) 

# tensorboard
tb = SummaryWriter(comment="My CNN")

# images, labels = next(iter(train_loader))
tb.add_graph(network, images)

# Training times
for epoch in range(10):
    train_loss = 0
    train_correct = 0
    
    for batch in train_loader:
        images, labels = batch
        images, labels = images.to(device), labels.to(device)
        
        # forward
        preds = network(images)
        
        # Compute the loss value.
        loss = loss_function(preds, labels)
        train_loss += loss.item()
        train_correct += get_num_correct(preds, labels)
        
        # Update the weights
        optimizer.zero_grad()  # Initialize to zero.
        loss.backward()
        optimizer.step()       # Updating the weights
        
        
    # Compute the average loss and accuracy.
    train_loss /= len(train_set)
    train_correct /= float(len(train_set))
    
    # Evaluate the model on the test set.
    test_loss, test_correct = test_model(network, loss_function, test_loader, device)
    
    # Record loss and accuracy values on the training and test sets.
    tb.add_scalars('loss', {'train': train_loss, 'test': test_loss}, epoch)
    tb.add_scalars('accuracy', {'train': train_correct, 'test': test_correct}, epoch)
    
    # Report progress
    print(f'#{epoch}: loss_test={test_loss:.4f}, loss_train={train_loss:.4f}, acc_test={test_correct:.4f}, acc_train={train_correct:.4f}')
    
    
# Don't forget it!
tb.close()

#0: loss_test=0.0013, loss_train=0.0014, acc_test=0.9611, acc_train=0.9583
#1: loss_test=0.0013, loss_train=0.0013, acc_test=0.9621, acc_train=0.9602
#2: loss_test=0.0012, loss_train=0.0012, acc_test=0.9624, acc_train=0.9627
#3: loss_test=0.0011, loss_train=0.0012, acc_test=0.9658, acc_train=0.9645
#4: loss_test=0.0010, loss_train=0.0011, acc_test=0.9705, acc_train=0.9661
#5: loss_test=0.0011, loss_train=0.0010, acc_test=0.9626, acc_train=0.9682
#6: loss_test=0.0011, loss_train=0.0010, acc_test=0.9659, acc_train=0.9691
#7: loss_test=0.0009, loss_train=0.0009, acc_test=0.9709, acc_train=0.9710
#8: loss_test=0.0009, loss_train=0.0009, acc_test=0.9717, acc_train=0.9715
#9: loss_test=0.0008, loss_train=0.0009, acc_test=0.9738, acc_train=0.9732
